## Analysis Notebook - differential Gene Expression Analysis

This notebook generates the sex-biased differential gene expression analysis.   Differential Analysis (DE) was performed using voom (Law et.al., 2014) with gene expression counts with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma. 

Within each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression:


           y = B0 + B1 sex + epsilon (error)
           
where y is the gene expression to be modeled sex denotes the reported sex of the subject.   The function named `fit_tissue()` performs this analysis and accepts two arguments, the `tissue` and an `object` and create the **model matrix** based  that tissue's sex. We will perform a linear fit after calculating normal factors (based on the library size) and calculate the dispersion using `voom` (mean variance model of dispersion). We are saving the resulting matrixes as files.

### 1. Data files created by this notebook

Output text files are written to the ``data/`` directory (at the same level as the ``jupyter`` directory). 

For each of the 39 tissues, this notebook produces the following results:

1. **{tissue}_DGE.csv**: topTable results for the edgeR/Limma differential analysis
2. **{tissue}_DGE_ensg_map.csv**: a convenience mapping of the ENSG to the geneSymbol
3. **{tissue}_DGE_refined.csv**: a convenience mapping of the topTable results satisfying the 1.5 fold change and adjusted P-Value < 0.05.

Additionally, diagnostic plots are produced:

1. **{tissue}-gene-y-voom-MDSplot-100.pdf**: multi-dimensional scaling plot (MDSplot), `red` `m` for the male and `blue` `f` voom variance model.
2. **{tissue}-gene-y-MDSplot-100.pdf**: MDSplot without voom.

### 1.1 load dependencies

In [1]:
suppressWarnings({suppressMessages({
options(warn = -1) 
library(gprofiler2)
library(downloader)
    library(readr)
    library(edgeR)
    library(limma)
    library(statmod)
    library(snakecase)
    library(limma)
    library(multtest)
    library(stringi)
    library(snakecase)
    library(dplyr)
    Sys.setenv(TAR = "/bin/tar") # for gzfile
})})

### 1.2 load corrected GTEx expressionSet object

In [2]:
message("\nReading GTEx expressionSet obj from ../data/gtex.corrected.rds\n")
obj <- readRDS(file = "data/gtex.corrected.rds")
pData(obj)$SAMPID <- gsub('-','\\.',pData(obj)$SAMPID)
message("\ndone reading GTEx corrected expressionSet Object\n")
dim(obj)


Reading GTEx expressionSet obj from ../data/gtex.corrected.rds



done reading GTEx corrected expressionSet Object




Features  Samples 
   55878    17382

### 2. Preparation for Differential Expression Analysis


### 2.1 Keep only the reduced tissues

In [3]:
tissue_reduction <- read.table("assets/tissues.tsv", header=TRUE, sep="\t",
                               skipNul=FALSE, stringsAsFactors = FALSE)
colnames(tissue_reduction)  <- c("SMTSD","female","male","include","display_name")

## only keep those we wish to include
tissue_reduction <- tissue_reduction[tissue_reduction$include==1,]
glimpse(tissue_reduction)

Rows: 39
Columns: 5
$ SMTSD        <chr> "adipose_subcutaneous", "adipose_visceral_omentum", "adr…
$ female       <int> 218, 170, 101, 153, 94, 209, 63, 58, 67, 74, 56, 54, 55,…
$ male         <int> 445, 371, 157, 279, 146, 454, 183, 157, 174, 181, 153, 1…
$ include      <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ display_name <chr> "Adipose (sc)", "Adipose (v)", "Adrenal gland", "Aorta",…


In [4]:
# convert the GTEx object via snakecase the name of the tissue to match
levels(pData(obj)$SMTSD)

[1] "Adipose - Subcutaneous"                   
 [2] "Adipose - Visceral (Omentum)"             
 [3] "Adrenal Gland"                            
 [4] "Artery - Aorta"                           
 [5] "Artery - Coronary"                        
 [6] "Artery - Tibial"                          
 [7] "Bladder"                                  
 [8] "Brain - Amygdala"                         
 [9] "Brain - Anterior cingulate cortex (BA24)" 
[10] "Brain - Caudate (basal ganglia)"          
[11] "Brain - Cerebellar Hemisphere"            
[12] "Brain - Cerebellum"                       
[13] "Brain - Cortex"                           
[14] "Brain - Frontal Cortex (BA9)"             
[15] "Brain - Hippocampus"                      
[16] "Brain - Hypothalamus"                     
[17] "Brain - Nucleus accumbens (basal ganglia)"
[18] "Brain - Putamen (basal ganglia)"          
[19] "Brain - Spinal cord (cervical c-1)"       
[20] "Brain - Substantia nigra"                 
[21] "Breast - Mammary Tissue"                  
[22] "Cells - Cultured fibroblasts"             
[23] "Cells - EBV-transformed lymphocytes"      
[24] "Cells - Leukemia cell line (CML)"         
[25] "Cervix - Ectocervix"                      
[26] "Cervix - Endocervix"                      
[27] "Colon - Sigmoid"                          
[28] "Colon - Transverse"                       
[29] "Esophagus - Gastroesophageal Junction"    
[30] "Esophagus - Mucosa"                       
[31] "Esophagus - Muscularis"                   
[32] "Fallopian Tube"                           
[33] "Heart - Atrial Appendage"                 
[34] "Heart - Left Ventricle"                   
[35] "Kidney - Cortex"                          
[36] "Kidney - Medulla"                         
[37] "Liver"                                    
[38] "Lung"                                     
[39] "Minor Salivary Gland"                     
[40] "Muscle - Skeletal"                        
[41] "Nerve - Tibial"                           
[42] "Ovary"                                    
[43] "Pancreas"                                 
[44] "Pituitary"                                
[45] "Prostate"                                 
[46] "Skin - Not Sun Exposed (Suprapubic)"      
[47] "Skin - Sun Exposed (Lower leg)"           
[48] "Small Intestine - Terminal Ileum"         
[49] "Spleen"                                   
[50] "Stomach"                                  
[51] "Testis"                                   
[52] "Thyroid"                                  
[53] "Uterus"                                   
[54] "Vagina"                                   
[55] "Whole Blood"

In [5]:
pData(obj)$SMTSD <- snakecase::to_snake_case(as.character(pData(obj)$SMTSD))
head(pData(obj)$SMTSD)

[1] "adipose_subcutaneous"     "muscle_skeletal"         
[3] "artery_tibial"            "artery_coronary"         
[5] "heart_atrial_appendage"   "adipose_visceral_omentum"

In [6]:
keep <- pData(obj)$SMTSD %in% tissue_reduction$SMTSD
table(keep)

keep
FALSE  TRUE 
 1851 15531 

In [7]:
obj   <-  obj[,keep == TRUE]
message("\nthe expressionSet object\n")
dim(obj)


the expressionSet object




Features  Samples 
   55878    15531

### 3. Differential analysis with edgeR and Limma

Using edgeR expressionSet object for the GTEx data set, perform a linear fit after calculating normal factors (based on the library size) and calculate the dispersion using `voom` (mean variance model of dispersion). We are saving the resulting matrixes as files.

### 3.1 Function fit_tissue

Function named `fit_tissue()`that accepts two arguments, the `tissue` and an `object` and create the **model matrix** based  that tissue's sex.

In [23]:
fit_tissue <- function (tissue, obj) {
    tissue_true             <- pData(obj)$SMTSD == tissue
    tissue_obj              <- obj[,tissue_true ==TRUE]
    tissue_sex              <- factor(pData(tissue_obj)$SEX)
    tissue_design           <- model.matrix(~tissue_sex)
    colnames(tissue_design) <- c("intercept","sex")

    
    # female
    female_obj           <- tissue_obj[,pData(tissue_obj)$SEX == 2]
    female_exprs_rowSums <- rowSums(cpm(exprs(female_obj))>=1)
    count_threshold     <- 0.25 * dim(female_obj)[2]
   
    # male
    male_obj           <- tissue_obj[,pData(tissue_obj)$SEX == 1]
    male_exprs_rowSums <- rowSums(cpm(exprs(male_obj))>=1)
    

    keep_male          <- male_exprs_rowSums >= count_threshold
    keep_female          <- female_exprs_rowSums >= count_threshold
    
    # now keep only those events that are meet either the male or the female criteria
    keep <- keep_male  & keep_female
    
    tissue_obj <- tissue_obj[keep==TRUE,]
    rm(male_obj)
    rm(female_obj)
    
    
    y_tissue       <- DGEList(counts=exprs(tissue_obj), group=tissue_sex)
    y_tissue       <- calcNormFactors(y_tissue)
    y_tissue_voom  <- voom(y_tissue, tissue_design)
    
    sex            <- ifelse(pData(tissue_obj)$SEX==1,'male','female')
    Gender         <- substring(sex,1,1)
    filename       <- paste0(paste0("pdf/", snakecase::to_snake_case(tissue)),"-gene-y-MDSplot-100.pdf")
    pdf (filename)
        plotMDS(y_tissue, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
    dev.off()
    filename       <- paste0(paste0("pdf/", snakecase::to_snake_case(tissue)),"-gene-y-voom-MDSplot-100.pdf")
    pdf (filename)    
        plotMDS(y_tissue_voom, labels=Gender, top=100, col=ifelse(Gender=="m","blue","red"), 
                gene.selection="common")
    dev.off()

    fit_tissue      <- lmFit(y_tissue_voom, tissue_design)
    fit_tissue      <- eBayes(fit_tissue, robust=TRUE)
    results_tissue  <- topTable (fit_tissue, coef='sex', number=nrow(y_tissue))
    results_refined <- results_tissue$adj.P.Val <= 0.05 & abs(results_tissue$logFC) >= abs(log2(1.5))
    
    
    filename  = paste(paste("data",gsub(" ","",tissue), sep="/"),"DGE.csv", sep="_")
    rfilename = paste(paste("data",gsub(" ","",tissue), sep="/"),"DGE_refined.csv", sep="_")
   
    write.table(results_tissue, filename, sep=',', quote=FALSE)
    write.table(results_tissue[results_refined,], rfilename, sep=',', quote=FALSE)
    return (results_tissue)
}

In [20]:
pData(obj)$SMTSD <- factor(pData(obj)$SMTSD)
levels(pData(obj)$SMTSD)
length(levels(pData(obj)$SMTSD))

[1] "adipose_subcutaneous"                 
 [2] "adipose_visceral_omentum"             
 [3] "adrenal_gland"                        
 [4] "artery_aorta"                         
 [5] "artery_coronary"                      
 [6] "artery_tibial"                        
 [7] "brain_caudate_basal_ganglia"          
 [8] "brain_cerebellar_hemisphere"          
 [9] "brain_cerebellum"                     
[10] "brain_cortex"                         
[11] "brain_frontal_cortex_ba_9"            
[12] "brain_hippocampus"                    
[13] "brain_hypothalamus"                   
[14] "brain_nucleus_accumbens_basal_ganglia"
[15] "brain_putamen_basal_ganglia"          
[16] "brain_spinal_cord_cervical_c_1"       
[17] "breast_mammary_tissue"                
[18] "cells_cultured_fibroblasts"           
[19] "cells_ebv_transformed_lymphocytes"    
[20] "colon_sigmoid"                        
[21] "colon_transverse"                     
[22] "esophagus_gastroesophageal_junction"  
[23] "esophagus_mucosa"                     
[24] "esophagus_muscularis"                 
[25] "heart_atrial_appendage"               
[26] "heart_left_ventricle"                 
[27] "liver"                                
[28] "lung"                                 
[29] "muscle_skeletal"                      
[30] "nerve_tibial"                         
[31] "pancreas"                             
[32] "pituitary"                            
[33] "skin_not_sun_exposed_suprapubic"      
[34] "skin_sun_exposed_lower_leg"           
[35] "small_intestine_terminal_ileum"       
[36] "spleen"                               
[37] "stomach"                              
[38] "thyroid"                              
[39] "whole_blood"

[1] 39

### 3.2 Looping through reduced Tissue set

The gprofiler function, mapping `ENSG` ids to the geneSymbol name will sometimes not have a match resulting in the message 

`No results to show
Please make sure that the organism or namespace is correct`

In [24]:

for (tissue in levels(pData(obj)$SMTSD))
    
    fit_tissue(tissue = tissue,obj = obj)



### Appendix - Metadata

For replicability and reproducibility purposes, we also print the following metadata:

### Appendix - 1. Checksums with the sha256 algorithm
1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

In [11]:
figure_id   = "differentialGeneExpression"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

Generating sha256 checksums of the artefacts in the `..data/` directory .. 

Warning message in system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", :
“running command 'cd ../data && find . -type f -exec sha256sum {} \;  >  ../metadata/differentialGeneExpression_sha256sums.txt' had status 1”


character(0)
attr(,"status")
[1] 1
attr(,"errmsg")
[1] "Resource temporarily unavailable"

Done!




ERROR: Error in data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), : File '../metadata/differentialGeneExpression_sha256sums.txt' does not exist or is non-readable. getwd()=='/Users/karleg/Downloads'


### Appendix - 2. Library Session Information

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]